# NJ Transit + Amtrak (NEC) Rail Performance Business Solution

The busiest passenger rail line in the United States, Amtrak also operates passenger rail service; together the NJ Transit and Amtrak operate nearly 750 trains across the NJ Transit rail network. Our data originates from here; https://www.kaggle.com/pranavbadami/nj-transit-amtrak-nec-performance. This data is organized into a monthly performance report of nearly every train trip on the rail network with the features; date, train ID, stop sequence, station train is traveling from, station ID of train origin, station train is traveling to, train destination ID, scheduled time, actual time, and delay in minutes. We will clean and analyze these features for further machine learning.

The business solution proposed is to create a classification model that can predict when a train is going to be late or not based on the features given. Clustering can also be used to segment rail lines that are in need of increased infrastructure and attention to upgrades to better serve the rail lines and prevent congestion and commute time. The purpose would be to better inform riders in advance of potential delays and cancellation using data driven machine learning algorithms. This would decrease congestion at rail stations where delayed crowds can often stagnate and minimize customer complaints and refunds by giving timelier notice of potential issues. 

I will be using various models and learning methods such as supervised learning (logistic regression, gradient boosting, KNN classifier, decision tree, random forest, SelectKBest, PCA, GridSearchCV hyperparameter tuning), unsupervised learning (t-SNE, PCA, KMeans and DBSCAN clustering) and deep learning models to implement the business solution above.


In [ ]:
from portfolio_utils import set_seed
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
set_seed(42)

# Load data if not already loaded (run this cell first)
try:
    df
except NameError:
    from portfolio_utils.data_loader import load_nj_transit
    df = load_nj_transit()
df.head(2)


# Exploratory Data Analysis and Data Cleaning

In [ ]:
# Examining the first five rows by calling pandas head function.
df.head()

,date,train_id,stop_sequence,from,from_id,to,to_id,scheduled_time,actual_time,delay_minutes,status,line,type
0,2020-05-01,5543,1.0,Newark Penn Station,107,Newark Penn Station,107,2020-05-01 23:38:00,2020-05-01 23:40:09,2.150000,departed,Raritan Valley,NJ Transit
1,2020-05-01,5543,2.0,Newark Penn Station,107,Union,38105,2020-05-01 23:47:00,2020-05-01 23:47:01,0.016667,departed,Raritan Valley,NJ Transit
2,2020-05-01,5543,3.0,Union,38105,Roselle Park,31,2020-05-01 23:50:00,2020-05-01 23:51:04,1.066667,departed,Raritan Valley,NJ Transit
3,2020-05-01,5543,4.0,Roselle Park,31,Cranford,32,2020-05-01 23:55:00,2020-05-01 23:55:31,0.516667,departed,Raritan Valley,NJ Transit
4,2020-05-01,5543,5.0,Cranford,32,Westfield,155,2020-05-01 23:59:00,2020-05-01 23:59:01,0.016667,departed,Raritan Valley,NJ Transit


There are a few features of interest here, most notable the delay in minutes column which seems to be a derivative of the scheduled time and actual time features. To_id and from_id correlates to the station names in the to and from columns. The date column spans the month of May

In [ ]:
# Examining DataFrame features data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98698 entries, 0 to 98697
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            98698 non-null  object 
 1   train_id        98698 non-null  object 
 2   stop_sequence   87172 non-null  float64
 3   from            98698 non-null  object 
 4   from_id         98698 non-null  int64  
 5   to              98698 non-null  object 
 6   to_id           98698 non-null  int64  
 7   scheduled_time  87172 non-null  object 
 8   actual_time     98698 non-null  object 
 9   delay_minutes   87172 non-null  float64
 10  status          98698 non-null  object 
 11  line            98698 non-null  object 
 12  type            98698 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 9.8+ MB


Data types seem to be correct for all features, we will further clean our data by dropping unneccesary columns.

In [ ]:
# Dropping to_id and from_id columns from our DataFrame.
df = df.drop(columns='to_id')
df = df.drop(columns='from_id')

Dropping redundant features such as to_id and from_id since our actual locations are listed in features to and from and can be used as dummy variables for easier interpretation. These will also not be useful for analysis.

In [ ]:
# Calling pandas .duplicated function to create boolean series of duplicate values and storing in a pandas DataFrame.
duplicate_rows_df = df[df.duplicated()]
print(duplicate_rows_df.shape)

(0, 11)


Checking DataFrame for rows containing duplicate data, we have only 11 duplicate rows out of 98,000 rows a negligible amount.

In [ ]:
# Finding the percent of missing values by calling pandas .isna function which returns a mask of bool values for each element in 
# DataFrame that indicates whether an element is not an NA value, and rounding the mean.
df.isna().mean().round(4) * 100

date               0.00
train_id           0.00
stop_sequence     11.68
from               0.00
to                 0.00
scheduled_time    11.68
actual_time        0.00
delay_minutes     11.68
status             0.00
line               0.00
type               0.00
dtype: float64

Checking the percentage of missing data in each feature. Stop_sequence, scheduled_time and delay_minutes need to have their missing values dropped for further machine learning. 
These features are missing a significant amount of data, but for the most part our dataset is complete. We will drop the rows that are missing to save the intact information within them.

In [ ]:
# .dropna returns the DataFrame with missing values dropped.
df = df.dropna()

In [ ]:
# Confirming removal of missing values from DataFrame.
df.isna().mean().round(4) * 100

date              0.0
train_id          0.0
stop_sequence     0.0
from              0.0
to                0.0
scheduled_time    0.0
actual_time       0.0
delay_minutes     0.0
status            0.0
line              0.0
type              0.0
dtype: float64

Now that we have cleaned our data we can go ahead and visualize some of the features of interest.

In [ ]:
# Creating distribution plot with seaborn for 'delay_minutes' column.
sns.histplot(df['delay_minutes'], kde=True)


This plot shows the distribution of our delay in minutes. Most delays were minor, with the highest density either having no delay or about a 5 minute delay. We will use this feature to create our target variable by creating a threshold that will alert to a delay greater than 15 minutes later.

In [ ]:
# Countplot for 'status' with seaborn.
sns.countplot(x ='status', data = df)

This count plot shows train status distribution. A count plot is a histogram across a categorical, instead of quantitative, variable. A majority of trains departed in a timely manner, the trains listed estimated status means they have not explicity departed and may be running late.
Canceled status indicates train has been canceled. We will one-hot encode these variables to run in our machine model.

In [ ]:
# Countplot displaying train 'line' distribution with seaborn.
plt.figure(figsize=(20,6))
sns.countplot(x ='line', data = df)

The majority of train lines are coming from the North Jersey coast, the Morristown line, and the Northeastern Corridor train lines. We have the smallest amount of data for the Princeton Shuttle line. We will also one-hot encode these variables to feed into our machine model and determine any lines of interest.

In [ ]:
# Defining figure size with pyplot.
plt.figure(figsize=(12, 6))
# Applying correlation scoring to dataframe.
corr = df.apply(lambda x: pd.factorize(x)[0]).corr()
# Creating heatmap with seaborn.
ax = sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, 
                 linewidths=.2, cmap="YlGnBu")

The strongest correlation being displayed is the correlation between train line and delay in minutes, as well as the correlation between to and from destinations and delay in minutes. 

## Time Series Analysis

Delay patterns are inherently temporal. We analyze **daily trend**, **day-of-week seasonality**, and **hour-of-day effects** to support scheduling, crew allocation, and proactive passenger alerts. All aggregations use the cleaned `df` (with non-null `delay_minutes` and `scheduled_time`) before dropping date/time columns for classification.

In [ ]:
# Time series view: ensure datetime, then aggregate by date
ts = df[["date", "scheduled_time", "delay_minutes"]].copy()
ts["date"] = pd.to_datetime(ts["date"], errors="coerce")
ts = ts.dropna(subset=["date", "delay_minutes"])
# Parse scheduled_time for hour (handle mixed formats)
ts["scheduled_dt"] = pd.to_datetime(ts["scheduled_time"], errors="coerce")
ts["hour"] = ts["scheduled_dt"].dt.hour
ts["day_of_week"] = ts["date"].dt.dayofweek  # 0=Monday

daily = ts.groupby("date")["delay_minutes"].agg(["mean", "median", "count"]).reset_index()
daily["rolling_mean_7"] = daily["mean"].rolling(7, min_periods=1).mean()

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
axes[0].plot(daily["date"], daily["mean"], alpha=0.6, label="Daily mean delay (min)")
axes[0].plot(daily["date"], daily["rolling_mean_7"], color="C1", lw=2, label="7-day rolling mean")
axes[0].set_ylabel("Delay (minutes)")
axes[0].set_title("Daily Average Delay and 7-Day Rolling Mean")
axes[0].legend(loc="upper right")
axes[0].grid(True, alpha=0.3)

axes[1].fill_between(daily["date"], daily["count"], alpha=0.5, color="steelblue")
axes[1].set_ylabel("Trip count")
axes[1].set_xlabel("Date")
axes[1].set_title("Daily Trip Volume")
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Day-of-week and hour-of-day seasonality
dow = ts.groupby("day_of_week")["delay_minutes"].mean().reset_index()
dow["day_name"] = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
hourly = ts.dropna(subset=["hour"]).groupby("hour")["delay_minutes"].mean().reset_index()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.barplot(data=dow, x="day_name", y="delay_minutes", color="steelblue", ax=axes[0])
axes[0].set_xlabel("Day of week")
axes[0].set_ylabel("Mean delay (minutes)")
axes[0].set_title("Mean Delay by Day of Week")
axes[0].tick_params(axis="x", rotation=45)

sns.lineplot(data=hourly, x="hour", y="delay_minutes", marker="o", ax=axes[1])
axes[1].set_xlabel("Hour of day (scheduled)")
axes[1].set_ylabel("Mean delay (minutes)")
axes[1].set_title("Mean Delay by Hour of Day")
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap: day of week x hour of day (mean delay in minutes)
pivot = ts.dropna(subset=["hour"]).pivot_table(
    values="delay_minutes", index="day_of_week", columns="hour", aggfunc="mean"
)
day_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
pivot.index = [day_labels[i] for i in pivot.index]
plt.figure(figsize=(14, 5))
sns.heatmap(pivot, cmap="YlOrRd", annot=False, fmt=".1f", cbar_kws={"label": "Mean delay (min)"})
plt.xlabel("Hour of day (scheduled)")
plt.ylabel("Day of week")
plt.title("Mean Delay by Day of Week and Hour of Day (Time Series View)")
plt.tight_layout()
plt.show()

**Time series takeaways:** Daily rolling mean smooths noise and can inform schedule padding. Day-of-week and hour-of-day patterns support crew and resource planning; the heatmap highlights peak-delay windows for targeted interventions. For production forecasting, consider `TimeSeriesSplit` for validation and autoregressive or lag-based features.

In [ ]:
# Dropping the scheduled time and actual time columns since we are interested in the amount of delay which is measured in the delay feature having been already extracted for us.
df = df.drop(columns='scheduled_time')
df = df.drop(columns='actual_time')
df = df.drop(columns='date')

# We can drop our type feature as after dropping missing values it only contains one unique value and won't contribute to our DataFrame.
df = df.drop(columns='type')

In [ ]:
# Creating new column for our target variable that consists of booleans for 'delay_minutes' values greater than 15.
df['late'] = df['delay_minutes']>15

In [ ]:
# Dropping 'delay_minutes' column from DataFrame.
df = df.drop(columns='delay_minutes')

We can now drop our delay column as we have extracted the necessary info from it by creating a target variable derived from it.

# Supervised Learning with SelectKBest Feature Selection

In [ ]:
# Create our target variable y.
y = df['late']

# Create X values by dropping our target.
X = df.drop(columns=['late'])

# Use pandas get_dummies function to create dummy variables necessary for statistical interpretation (One Hot Encoding).
X = pd.get_dummies(X)

In [ ]:
# Creating a countplot for our target variable y with seaborn.
sns.countplot(y)

Examining the distribution of our target variable to see if our classes are balanced for best performance in supervised machine learning models.
The countplot shows that the target class is imbalanced, with our target value of delay greater than 15 minutes having as expected less True values. We must fix this with SMOTE (Synthetic Minority Oversampling Technique). SMOTE synthetically oversamples the minority target class to match when sampling strategy is set to minority.

In [ ]:
# Creating oversampler with minority sampling strategy.
sm = SMOTE(sampling_strategy='minority', random_state=42)

# Fitting SMOTE to X and y variables.
X, y = sm.fit_resample(X, y)

In [ ]:
# Creating another countplot for our target variable y with seaborn.
sns.countplot(y)

Re-examining y distribution to confirm classes have been balanced by SMOTE. Our distribution of True values now matches that of False values.

In [ ]:
# Putting X variable back into DataFrame after resampling for feature selection.
X = pd.DataFrame(X)

# Fitting SelectKBest feature selector to our variables using f_classif for our classification task with a k-value of 200.
selector = SelectKBest(f_classif, k=200).fit(X, y)

We will select the 200 best features based on highest k scores to reduce runtime and retain information. Reducing k-value to a number that is too low will cause overfitting due to loss of too much information. If we set a k value that is too high it will not reduce dimensionality therefore no impact on performance.

In [ ]:
# Creating boolean values by calling get_support on selector.
boolean = selector.get_support()

# Pulling newly selected columns into a new X variable with booleans.
X_new = X[X.columns[boolean]]

Our new X variable now consists of our 200 selected best features based on K values.

In [ ]:
# Checking shape property of original X variable.
X.shape

In [ ]:
# Checking shape property of new X variable.
X_new.shape

Examining newly selected feature set and confirming dimensionality reduction. We've successfully reduced our DataFrame by 590 columns, this will dramatically reduce run time.

In [ ]:
# Standardize features by removing the mean and scaling to unit variance with sklearns StandardScaler.
scaler = StandardScaler()

# Fit scaler to data by calling fit_transform on our new data.
scaled_data = scaler.fit_transform(X_new)

Scaling newly transformed features for machine learning model processing.

In [ ]:
# Using sklearns train_test_split to split data into random train and test subsets.
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.20, stratify=y, random_state=42)

Now that we have finished processing, cleaning, and selecting our data features we are able to feed them into our supervised learning machine models and measure their performance.

## Logistic Regression Classifier with SelectKBest Selected Features

In [ ]:
# Timing our model.
start_time = time.time()

# Adjust our max iteration value for proper convergence.
clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)

# Print model time. 
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Setting model run timer to print time upon completion, while adjusting the max iteration value for proper convergence. 

In [ ]:
# Print cross validation score.
cross_val_score(clf, X_train, y_train, cv=10)

The cross validation score for the Logistic Regression Classifier is consistent and doesn't show any signs of over or underfitting.

In [ ]:
# Print classification report for our Logistic Regression classifier.
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Classification report:")
print(report)
print("Confusion matrix:")
print(cm)

The classification report for our Logistic Regression classifier is showing consistent values and does not show signs of fitment issues. Logistic Regression classification performs best on binary outputs so it is not surprising this model works okay on our data. Performance has much room for improvement however.

## Gradient Boosting Classifier with SelectKbest Feature Selection

In [ ]:
# Timing our model.
start_time = time.time()

# Fitting gradient boosting classifier to training data. 
clf = GradientBoostingClassifier().fit(X_train, y_train)

# Print model run time.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

In [ ]:
# Print cross validation score.
cross_val_score(clf, X_train, y_train, cv=5)

The cross validation score for the Gradient Boosting Classifier is consistent in the .88 range and doesn't show any signs of over or underfitting. This means our model should be expected to handle new data, since it is not overtraining on the training data. 

In [ ]:
# Print classification report for our Gradient Boosting classifier.
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Classification report:")
print(report)
print("Confusion matrix:")
print(cm)

The classification report for our Gradient Boosting Classifier is also showing consistent values and does not show evidence of fitment issues. We are seeing slightly higher precision here than for our previous logistic regression model although at double the run time. Additional research could include hyperparameter tuning the learning rate and number of estimators to improve accuracy or run time since we did not include any hyperparameters.

## KNN Classifier with SelectKBest Feature Selection

In [ ]:
# Timing our model.
start_time = time.time()

# Creating KNN classifier with n_neighbors set to 25 to establish performance. 
knn = KNeighborsClassifier(n_neighbors=25)

# Fitting KNN classifier to training data. 
knn.fit(X_train, y_train)

print("--- %s seconds ---" % round(time.time() - start_time, 3))

Model run time is much better than all previous models. n_neighbors hyperparameter was set to 25 as a base parameter, this may be tuned in further research to optimize KNN classification.

In [ ]:
# Print cross validation score for training variables.
cross_val_score(knn, X_train, y_train, cv=10)

The cross validation score for the KNN Classifier is consistent and doesn't show any signs of over or underfitting.

In [ ]:
# Print classification report.
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Classification report:")
print(report)
print("Confusion matrix:")
print(cm)

The classification report for our KNN Classifier is also showing consistent values and does not show evidence of fitment issues. Weighted averages are also higher as well as F1 score. We are seeing much better model performance here in all aspects than the previous gradient boosting and logistic regression models even without any hyperparameter tuning. KNN classification works well on dimensionally reduced data and runs fast as it does not require any prior training also known as lazy learning. Lazy learning is a machine learning method in which generalization of the training data is delayed until a query is made to the system, as opposed to eager learning, where the system tries to generalize the training data before receiving queries. This reduces the computational complexity needed to run the model.

## Decision Tree Classifier with SelectKBest Selected Features and Optimized Hyperparameters

In [ ]:
# Timing our model.
start_time = time.time()

# Building tree.
decision_tree = tree.DecisionTreeClassifier()

# Fit tree to training data.
decision_tree.fit(X_train, y_train)

# Print model run time.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

We immediately see that decision tree model run time is much better than all previous models and half the time of our KNN classification model even without any hyperparameter tuning which we will perform in the followin cells. Binary decision trees are made through a process known as binary recursive partitioning. This is an iterative process of splitting the data into partitions, and then splitting it up further on each of the branches. This process also helps clean up unnecessary features and results in a fast and inexpensive model to run with performance equal to other classification techniques that require more compuational intensity.

In [ ]:
# Create tree parameter grid adjusting combination of criterion, max_depth, and max_features for best model performance.
tree_para = {'criterion':['gini','entropy'],
             'max_depth':[5,10,50,100,500],
             'max_features':[5,10,50,100,200]}
          
# Exhaustive search over specified parameter values using GridSearchCV. GridSearchCV will score all possible combinations of hyperparameters.
clf = GridSearchCV(decision_tree, tree_para, cv=5)

# Fit tree to training data using GridSearchCV.
clf.fit(X_train, y_train)

# Append GridSearchCV results to new DataFrame for easier interpretation.
gscv_df = pd.DataFrame(clf.cv_results_)

# Sort values by test score rank to find hyperparameters with the highest score.
gscv_df.sort_values('rank_test_score')

Applying GridSearchCV to find optimal hyperparameters to improve our classification report. We are looking at a few parameters to see what works best with our model by implementing them into a parameter grid. More complex parameter grids require increased computation time. Finally we will be putting our results into a Pandas DataFrame for easier interpretation.

In [ ]:
# Start timing model.
start_time = time.time()

# Build decision tree using hyperparameters with the highest ranked test score.
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_depth=500,
    max_features=200,
)

# Fit tree to training data.
decision_tree.fit(X_train, y_train)

print("--- %s seconds ---" % round(time.time() - start_time, 3))

We see a slight improvement in run time with optimized hyperparameters reducing run time by half a second.

In [ ]:
# Print cross validation score.
cross_val_score(decision_tree, X_train, y_train, cv=10)

The cross validation score for the Decision Tree Classifier is consistent and doesn't show any signs of over or underfitting.

In [ ]:
# Print classificiation report. 
y_pred = decision_tree.predict(X_test)
dt_accuracy = accuracy_score(y_test,y_pred)
decision_tree_report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", dt_accuracy)
print("Classification report:")
print(decision_tree_report)
print("Confusion matrix:")
print(cm)

Model performance is much better than our previous models in all aspects with no signs of overfitting. We have reduced run time to 4.3 seconds and increased the accuracy score. Decision tree models are very fast and usually not computationally intensive as they do not compute all possibilities for a tree, and optimize which data to split in the given nodes. As the decision tree splits data further its speed increases also as it iterates over a smaller subset of data. 

## Random Forest Classifier with SelectKBest Selected Features and Optimized Hyperparameters

In [ ]:
# Timing our model.
start_time = time.time()

# Building random forest classifier.
rfc = ensemble.RandomForestClassifier()

# Fitting classifier to training data. 
rfc.fit(X_train, y_train)

# Print model runtime.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Setting up Random Forest Classifier by fitting to data and setting model run timer to print time upon completion. Model run time is much longer than the previous models. This is because random forest builds multiple decision trees and merges them together requiring more computational intensity than a single tree. We will try to reduce the run time with hyperparameter tuning below.

In [ ]:
# Creating hyperparameter grid.
param_grid = { 
    'n_estimators': [5,50,100,500,1000],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,6,8,10],
    'criterion' :['gini', 'entropy']
}

# Setting up GridSearchCV to run on our random forest classifier.
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

# Fitting GridSearchCV to training data. 
CV_rfc.fit(X_train, y_train)

We are looking at a few parameter combinations to see what works best with our model by implementing them into a parameter grid once again. More complex parameter grids require increased computation time.

In [ ]:
# Calling GridSearchCV .best_params_ to pull best parameters.
CV_rfc.best_params_

This time we will find the best parameters by calling '.best_params_' instead of sorting by test rank score in a DataFrame. This will pull the best parameters from GridSearchCV also.

In [ ]:
# Timing our model.
start_time = time.time()

# Building random forest classifier with best parameters from GridSearchCV.
rfc = ensemble.RandomForestClassifier(criterion='entropy', 
                                      max_depth=10,
                                      max_features='sqrt',
                                      n_estimators=5, random_state=42)
# Fitting RFC with new hyperparameters.
rfc.fit(X_train, y_train)

# Print model run time.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Creating our model using optimized hyperparameters from GridSearchCV. We see a dramatic decrease in model run time from 50 seconds to just over 1 second.

In [ ]:
# Print cross-validation score.
cross_val_score(rfc, X_train, y_train, cv=10)

Cross-validation score shows consistent scoring and no signs of overfitting or underfitting.

In [ ]:
# Print classification report.
y_pred = rfc.predict(X_test) 
rfc_accuracy = accuracy_score(y_test,y_pred)
rfc_report = classification_report(y_test, y_pred)
rfc_cm = confusion_matrix(y_test, y_pred)

print("Classification report:")
print("Accuracy: ", rfc_accuracy)
print(rfc_report)
print("Confusion matrix:")
print(rfc_cm)

Model performance is not as good as the previous decision tree model in all aspects; precision, recall, f1-score, accuracy, and their respective averages. Once again no signs of overfitting or underfitting. We significantly decreased our run time with hyperparameter tuning using GridSearchCV to perform better than our Decision Tree Classifier however with a lower scoring.

As shown here ensemble models are not always better performers than their individual counterparts, they are better if the single decision tree models suffer from instability most usually.

### Supervised Learning with SelectKBest: Conclusion

In conclusion, decision tree model performance was best out of all other models (logistic regression, gradient boosting, KNN, random forest) using SelectKBest selected features with the best model run time and performance metrics. I hypothesize that this is due to our dimensionality reduction technique SelectKBest that allowed our decision tree to perform better than our random forest model. Without feature selection we most likely would have seen the opposite effect.

# Supervised Learning with PCA Feature Decomposition

In [ ]:
# Create our target variable y.
y = df['late']

# Create X values by dropping our target.
X = df.drop(columns=['late'])

# Use pandas get_dummies function to create dummy variables necessary for statistical interpretation (One Hot Encoding).
X = pd.get_dummies(X)

In [ ]:
# Creating a countplot for our target variable y with seaborn.
sns.countplot(y)

Examining initial target class to check balancing.

In [ ]:
# Creating oversampler with minority sampling strategy.
sm = SMOTE(sampling_strategy='minority', random_state=42)

# Fitting SMOTE to X and y variables.
X, y = sm.fit_resample(X, y)

In [ ]:
# Creating another countplot for our target variable y with seaborn.
sns.countplot(y)

Class balancing using SMOTE and confirmation of class balance.

In [ ]:
# Standardize features by removing the mean and scaling to unit variance with sklearns StandardScaler.
scaler = StandardScaler()

# Fit scaler to data by calling fit_transform on our data.
scaled_data = scaler.fit_transform(X)

First we must scale our data so that it can be fit into our PCA model.

In [ ]:
# PCA feature reduction with the n_components set to represent 50% of the variance in our dataset.
sklearn_pca = PCA(n_components = 0.50)

# Apply PCA to our scaled data.
X_pca = sklearn_pca.fit_transform(scaled_data)

In [ ]:
# Checking shape property of original X variable.
X.shape

In [ ]:
# Checking shape property of new X variable.
X_pca.shape

Examing newly created PCA decomposed features. We successfully reduced DataFrame by 528 features to account for 50% of the variance in our original dataset.

In [ ]:
# Using sklearns train_test_split to split data into random train and test subsets.
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.20, stratify=y, random_state=42)

## Logistic Regression Classifier with PCA

In [ ]:
# Timing our logistic regression classifier model.
start_time = time.time()

# Adjusting max iteration value for proper convergence.
clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)

# Print model runtime.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

In [ ]:
# Print the cross validation score.
cross_val_score(clf, X_train, y_train, cv=10)

The cross validation score for the logistic regression classifier is showing consistent scoring and no signs of overfitting or underfitting.

In [ ]:
# Print classification report.
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Classification report:")
print(report)
print("Confusion matrix:")
print(cm)

The classification report for our logistic regression classifier shows no signs of overfitting. Recall for false values and precision for true values decreased performance compared to SelectKBest, but we are seeing performance improvement in the weighted averages of precision, recall, and f-1 score as well as a 1% improvement in accuracy score. Most remarkable is the runtime reduction from aroudn 50 seconds to around 10 seconds even though the number of dimensions actually increased with PCA feature decomposition (262 features for PCA versus 200 for SelectKBest).

## Gradient Boosting Classifier with PCA

In [ ]:
# Timing our model.
start_time = time.time()

# Fitting gradient boosting classifier to training data. 
clf = GradientBoostingClassifier().fit(X_train, y_train)

# Print model run time.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Timing our gradient boosting classifier, we have a significantly longer run time than the logistic regression classifier as well as our gradient boosting classifier that we used with SelectKBest. This is due to the higher dimensionality of the PCA data so the gradient boosting classifier must form more trees during modeling.

In [ ]:
# Print cross validation score.
cross_val_score(clf, X_train, y_train, cv=5)

Cross validation score for Gradient Boosting Classifier is consistent and shows no signs of overfitting or underfitting.

In [ ]:
# Print classification report.
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Classification report:")
print(report)
print("Confusion matrix:")
print(cm)

We have slight performance increase of about .02 percent in weighted averages in all areas compared to logistic regression but at the cost of a much longer run time. The increased dimensionality of our PCA data also improved scoring in comparison to SelectKBest data but also increasing run time by almost 10x. The trade off in run time for a small performance increase seems to make this model an unlikely choice.

## KNN Classifier with PCA

In [ ]:
# Timing our model.
start_time = time.time()

# Creating KNN classifier with n_neighbors set to 25 to establish performance. 
knn = KNeighborsClassifier(n_neighbors=25)

# Fitting KNN classifier to training data. 
knn.fit(X_train, y_train)

# Print model runtime.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

In [ ]:
# Print cross-validation score.
cross_val_score(knn, X_train, y_train, cv=10)

Cross validation score is consistent and shows no signs of fitment issues.

In [ ]:
# Printing classification report with test variables and prediction.
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Classification report:")
print(report)
print("Confusion matrix:")
print(cm)

Classification report for KNN Classifier is much better than our previous models with a higher weighted average in all categories. KNN was able to process PCA reduced data in a faster time and with better scoring than SelectKBest selected features despite having a higher dimensionality.

## Decision Tree Classifier with PCA Decomposed Features and Optimized Hyperparameters

In [ ]:
# Timing our model.
start_time = time.time()

# Building tree.
decision_tree = tree.DecisionTreeClassifier()

# Fit tree to training data.
decision_tree.fit(X_train, y_train)

# Print model run time.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Timing and training our initial tree before running GridSearchCV. We have a signficant increase in run time compared to our decision tree classifier with SelectKBest selected features due to increased dimensionality.

In [ ]:
# Create parameter grid for GridSearchCV.
tree_para = {'criterion':['gini','entropy'],
             'max_depth':[10,100,500,1000],
             'max_features':[10,50,100,200]}

# Create decision tree classifier with GridSearchCV.
clf = GridSearchCV(decision_tree, tree_para, cv=5)

# Fitting decision tree classifier to training data.
clf.fit(X_train, y_train)

# Putting GridSearchCV results into Pandas DataFrame.
gscv_df = pd.DataFrame(clf.cv_results_)

# Sorting DataFrame by highest ranked test score.
gscv_df.sort_values('rank_test_score')

The best performance was seen using the hyperparameters criterion: gini, max_depth: 500, and max_features: 200. We will use these hyperparameters to train our next decision tree model.

In [ ]:
# Timing our model.
start_time = time.time()

# Creating decision tree using new hyperparameters.
decision_tree = tree.DecisionTreeClassifier(
    criterion='gini',
    max_depth=500,
    max_features=200,
)

# Fitting tree to training data.
decision_tree.fit(X_train, y_train)

# Print model runtime.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Timing and training our tree using best parameters from GridSearchCV, we were able to decrease the model run time by over 10 seconds. 

In [ ]:
# Print cross validation score.
cross_val_score(decision_tree, X_train, y_train, cv=10)

Our decision tree is showing consistent scoring with no signs of underfitting or overfitting.

In [ ]:
# Print classification report.
y_pred = decision_tree.predict(X_test)
dt_accuracy = accuracy_score(y_test,y_pred)
decision_tree_report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", dt_accuracy)
print("Classification report:")
print(decision_tree_report)
print("Confusion matrix:")
print(cm)

This decision tree model with PCA decomposed features is performing better now with a lower run time. However, in comparison to our decision tree classifier using SelectKBest features, model run time is about 10x longer. Accuracy score has only improved from .956 with SelectKBest to .982 with PCA decomposed features. So the decision tree has a trade off between processing time, dimensionality, and accuracy that must be established for each use case. It seems hard to justify the increase in run time for the small benefit of slight increase in accuracy scoring seen here.


## Random Forest Classifier with GridSearchCV and PCA

In [ ]:
# Timing our model.
start_time = time.time()

# Building random forest classifier.
rfc = ensemble.RandomForestClassifier()

# Fitting classifier to training data. 
rfc.fit(X_train, y_train)

# Print model runtime.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Creating and timing our initial model before processing with GridSearchCV to find the best hyperparameters.

In [ ]:
# Creating hyperparameter grid.
param_grid = { 
    'n_estimators': [5,50,100,500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,6,8],
    'criterion' :['gini', 'entropy']
}

# Setting up GridSearchCV to run on our random forest classifier.
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)

# Fitting GridSearchCV to training data. 
CV_rfc.fit(X_train, y_train)

In [ ]:
# Calling .best_params_ to pull best parameters from GridSearchCV.
CV_rfc.best_params_

In [ ]:
# Timing our model.
start_time = time.time()

# Building random forest classifier with best parameters from GridSearchCV.
rfc = ensemble.RandomForestClassifier(criterion='gini', 
                                      max_depth=8,
                                      max_features='sqrt',
                                      n_estimators=100, random_state=42)

# Fitting RFC with new hyperparameters.
rfc.fit(X_train, y_train)

# Printing model runtime.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Using optimized hyperparameters from GridSearchCV we were able to reduce run time by about 40%. 

In [ ]:
# Print cross-validation score.
cross_val_score(rfc, X_train, y_train, cv=10)

Cross-validation score is consistent and does not show any signs of overfitting or underfitting.

In [ ]:
# Print classification report.
y_pred = rfc.predict(X_test) 
rfc_accuracy = accuracy_score(y_test,y_pred)
rfc_report = classification_report(y_test, y_pred)
rfc_cm = confusion_matrix(y_test, y_pred)

print("Classification report:")
print("Accuracy: ", rfc_accuracy)
print(rfc_report)
print("Confusion matrix:")
print(rfc_cm)

The previous decision tree model with PCA decomposed features is performing better than the random forest model here in terms of both runtime and scoring. Random forest run time is longer with a reduction in accuracy score and weighted averages for precision, recall, and f1-score.  As shown again here ensemble models are not always better performers than their individual counterparts, they are better if the single decision tree models suffer from instability most usually.

In conclusion, our KNN classifier worked best using PCA feature reduced data with the best combination of run time and performance. Decision tree performed best in terms of scoring but at the cost of a long run time.

# Unsupervised Learning

We will use KMeans and DBSCAN clustering techniques on our data using PCA and t-SNE feature reduced data to see which combination of dimensionality reduction and clustering techniques works best for this dataset.

## PCA Feature Decomposition

In [ ]:
# df loaded above (re-run first load cell if needed)
df.head(2)


In [ ]:
# Dropping redundant columns 'to_id' and 'from_id' from DataFrame.
df = df.drop(columns='to_id')
df = df.drop(columns='from_id')

Dropping redundant features such as to_id and from_id since our actual locations are listed in features to and from and can be used as dummy variables.

In [ ]:
# Dropping missing values from DataFrame.
df = df.dropna()

In [ ]:
# Drop redundant columns once again from DataFrame.
df = df.drop(columns='scheduled_time')
df = df.drop(columns='actual_time')
df = df.drop(columns='date')

Dropping the scheduled time and actual time columns since we are only interested in the amount of delay which is measured and already extracted from these columns for us in the 'delay_minutes' feature. The date feature is also not important to our clustering model.

In [ ]:
# Dropping 'type' column, contains only one value.
df = df.drop(columns='type')

We can drop our type feature as after dropping missing values it only contains one unique value and won't contribute to our data.

In [ ]:
# Creating a new feature to indicate if a train is late or not, we will use a threshold of 15 minute delay time.
df['late'] = df['delay_minutes']>15

# We can now drop our delay column as we have extracted the necessary info from it.
df = df.drop(columns='delay_minutes')

# One-hot encoding statistical variables for machine learning.
df = pd.get_dummies(df, columns={'from','to','line','train_id','status','late'})

Before Principle Component Analysis, we must scale our features to bring the mean to zero and the standard deviation to 1.

In [ ]:
# Standardize features by removing the mean and scaling to unit variance with sklearns StandardScaler.
scaler = StandardScaler()

# Fit scaler to data by calling fit_transform on our data.
X_scale = scaler.fit_transform(df)

# Storing scaled data into new DataFrame.
df_scale = pd.DataFrame(X_scale, columns=df.columns)
df_scale.head()

In [ ]:
# PCA with the number of components set to 2, this will reduce our data to two dimensions.
pca = PCA(n_components=2)

# We get the components by calling fit_transform method with our data.
pca_components = pca.fit_transform(df_scale)

In [ ]:
# Defining pyplot figure size.
plt.figure(figsize=(10,5))

# Creating scatterplot of pca component dimensions with pyplot.
plt.scatter(pca_components[:, 0], pca_components[:, 1])
plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

This scatterplot shows what our data looks like now reduced to two dimensions, KMeans and DBSCAN will label these clusters according to their respective algorithms and we will test the performance of each using the silhouette score metric.

## Applying KMeans to PCA

In [ ]:
# Components are now in Pandas DataFrame.
PCA_components = pd.DataFrame(pca_components)
# Create range of k values.
ks = range(1, 10)
inertias = []
for k in ks:
    # Create KMeans instance with k clusters: model.
    model = KMeans(n_clusters=k)

    # Fit model to samples.
    model.fit(PCA_components.iloc[:,:3])
    
    # Append the inertia to the list of inertias.
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

The elbow method uses the K value plotted on the x-axis and the inertia plotted on the y-axis to choose where the plot comes to a point or 'elbow'. This visual method may give us an area to focus on but it is wise to go ahead and score all clusters up to 10 statistically. The elbow method here would give us a KMeans cluster value of 3.

In [ ]:
# Defining KMeans cluster number of 2 and labeling PCA components.
labels = KMeans(n_clusters=2, random_state=42).fit_predict(PCA_components)
# Print silhouette score of KMeans.
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=3, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

Elbow method score, let's see if this score remains the highest.

In [ ]:
labels = KMeans(n_clusters=4, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

The silhouette score seen here is very close to our elbow value for KMeans of 3 but still an improvement. If we only relied on visual methods we may have chose a KMeans value that did not perform as well.

In [ ]:
labels = KMeans(n_clusters=5, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=6, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=7, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=8, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=9, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=10, random_state=42).fit_predict(PCA_components)
print(metrics.silhouette_score(PCA_components, labels, metric='euclidean'))

In [ ]:
# Defining the k-means with the best silhouette value (4) and timing it.
kmeans_cluster = KMeans(n_clusters=4, random_state=42)
# Fitting and timing clustering.
%timeit kmeans_cluster.fit(PCA_components)
y_pred = kmeans_cluster.predict(PCA_components)

Timing and building the best performing model with a KMeans value of 4 and a silhouette score of .8. A perfect silhouette score would be a value of 1. 

## Cluster Analysis of KMeans Clusters using PCA

In [ ]:
# df loaded above (re-run first load cell if needed)
df.head(2)


Now that we have recreated our DataFrame before processing we can create a feature to label the data to its respective cluster.

In [ ]:
# Finally, attach the cluster labels from KMeans to the original DataFrame.
df2['cluster_label'] = kmeans_cluster.labels_

In [ ]:
# Examining DataFrame to confirm cluster labels have been attached.
df2.head()

In [ ]:
# Creating boolean values to create new cluster DataFrames.
clusters1 = df2['cluster_label'] == 0
clusters2 = df2['cluster_label'] == 1
clusters3 = df2['cluster_label'] == 2
clusters4 = df2['cluster_label'] == 3

In [ ]:
# Applying boolean against dataframe to pull correct rows.
clusters1df = df2[clusters1]
clusters2df = df2[clusters2]
clusters3df = df2[clusters3]
clusters4df = df2[clusters4]

Now that we have labeled our data with the correct cluster and put the clusters into an individual DataFrame, we can analyze and plot each cluster by analyzing the cluster DataFrames.

### First Cluster Analysis

In [ ]:
# Examining cluster size.
clusters1df.shape

The first cluster is the largest in comparison to the other four clusters with our PCA data using KMeans clustering. 

In [ ]:
# Creating countplot with seaborn and defining figure size.
plt.figure(figsize=(6,4))
sns.countplot(clusters1df['late'])

This cluster is comprised of mostly unlate trains with some late trains slipping through as shown here on the countplot for our late variable.

In [ ]:
# Defining figure size.
plt.figure(figsize=(10,6))
# Creating countplot of train lines for analysis using seaborn.
sns.countplot(clusters1df['line'], hue=clusters1df['late'])

This cluster seems to be representative of mostly unlate trains from various lines. We see some late values for some train lines in here but with counts significantly less than unlate trains. 

### Second Cluster Analysis

In [ ]:
# Examining cluster size.
clusters2df.shape

This is the second largest cluster in size compared to other four clusters.

In [ ]:
# Creating countplot of our late variable with seaborn and defining figure size.
plt.figure(figsize=(10,6))
sns.countplot(clusters2df['late'])

This second cluster is also comprised of mostly unlate trains with a few late values slipping in like before in the first cluster. This is not surprising considering the size of this cluster as it is almost half the dataset like the first.

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(clusters2df['line'], hue=clusters2df['late'])

This cluster is comprised mostly of different train lines than the first. We have some very small amounts of some train lines from the first cluster like the Morristown line. 

### Third Cluster Analysis

In [ ]:
# Examining cluster size.
clusters3df.shape

This cluster size is much smaller than our first two clusters with only 1,124 values.

In [ ]:
# Creating countplot of our late variable with seaborn and defining figure size.
plt.figure(figsize=(10,6))
sns.countplot(clusters3df['late'])

Unlike the first two clusters also this cluster is entirely comprised of late trains with no false values.

In [ ]:
plt.figure(figsize=(10,6))
# Creating countplot of train lines for analysis using seaborn.
sns.countplot(clusters3df['line'], hue=clusters3df['late'])

Clustering algorith grouped only trains from one line into this cluster interestingly. This suggests the Atlantic City Line is in need of upgrades.

### Fourth Cluster Analysis

In [ ]:
# Examining cluster size.
clusters4df.shape

In [ ]:
# Creating countplot of our late variable with seaborn and defining figure size.
plt.figure(figsize=(10,6))
sns.countplot(clusters4df['late'])

In [ ]:
plt.figure(figsize=(10,6))
# Creating countplot of train lines for analysis using seaborn.
sns.countplot(clusters4df['line'], hue=clusters4df['late'])

It seems that the Montclair-Boonton and Pascack Valley lines are in need of upgrades also. Our clustering algorithm was able to group these lines from the other clusters.

In [ ]:
# Finally, let's visualize the clusters formed by KMeans and PCA with a simple scatterplot.
plt.figure(figsize=(10,6))
plt.scatter(PCA_components[0],PCA_components[1], c=y_pred); 
plt.show()

## Applying DBSCAN Clustering to PCA

In [ ]:
dbscan_cluster = DBSCAN(eps=.2, min_samples=150)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

Defining the density based algorithm DBSCAN and adjusting the epsilon and minimum sample values. The silhouette score may be improved upon with different hyperparameter values.

In [ ]:
dbscan_cluster = DBSCAN(eps=.1, min_samples=150)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

Reducing the epsilon value seems to have made our model perform slightly better, let's try increasing the minimum sample value instead.

In [ ]:
dbscan_cluster = DBSCAN(eps=.2, min_samples=200)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

Slight improvement on score with increased minimum sample value and using the initial epsilon value. Let's try to increase the epsilon value further.

In [ ]:
dbscan_cluster = DBSCAN(eps=.3, min_samples=200)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

Setting the EPS at .3 causes our kernel to crash with a large minimum sample value, so we will try further adjusting our minimum sample value.

In [ ]:
dbscan_cluster = DBSCAN(eps=.1, min_samples=250)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

Increased minimum sample size caused reduced performance for DBSCAN clustering here. Let's try a smaller sample size.

In [ ]:
dbscan_cluster = DBSCAN(eps=.1, min_samples=25)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

In [ ]:
dbscan_cluster = DBSCAN(eps=.1, min_samples=100)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

In [ ]:
dbscan_cluster = DBSCAN(eps=.2, min_samples=100)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

In [ ]:
dbscan_cluster = DBSCAN(eps=.3, min_samples=125)
labels = dbscan_cluster.fit_predict(pca_components)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(pca_components, labels, metric='euclidean')))

Hyperparameter tuning was able to increase model performance significantly adjusting minimum sample value and finally settling on a value of 125 and epsilon value of .3. Although silhouette score performance is not better than KMeans it may still provide meaningful clustering so we will analyze the clusters below.

## DBSCAN Cluster Analysis with PCA Dimensionality Reduction

In [ ]:
np.unique(labels)

Let's see how many clusters our DBSCAN algorithm was able to identify by calling unique values for labeled clustering, we have 9 clusters. The -1 value is where DBSCAN grouped outliers.

In [ ]:
df2['dbscan_pca_cluster_label'] = labels

Attaching DBSCAN cluster labels to original dataframe.

In [ ]:
cluster1 = df2['dbscan_pca_cluster_label'] == 0
cluster2 = df2['dbscan_pca_cluster_label'] == 1
cluster3 = df2['dbscan_pca_cluster_label'] == 2
cluster4 = df2['dbscan_pca_cluster_label'] == 3
cluster5 = df2['dbscan_pca_cluster_label'] == 4
cluster6 = df2['dbscan_pca_cluster_label'] == 5
cluster7 = df2['dbscan_pca_cluster_label'] == 6
cluster8 = df2['dbscan_pca_cluster_label'] == 7
cluster9 = df2['dbscan_pca_cluster_label'] == 8
outliercluster = df2['dbscan_pca_cluster_label'] == -1

Creating boolean values to create new cluster dataframes.

In [ ]:
cluster1df = df2[cluster1]
cluster2df = df2[cluster2]
cluster3df = df2[cluster3]
cluster4df = df2[cluster4]
cluster5df = df2[cluster5]
cluster6df = df2[cluster6]
cluster7df = df2[cluster7]
cluster8df = df2[cluster8]
cluster9df = df2[cluster9]
outlierclusterdf = df2[outliercluster]

Applying boolean against dataframe to pull correct rows.

### First Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster1df['line'], hue=cluster1df['late'])

### Second Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster2df['line'], hue=cluster2df['late'])

### Third Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster3df['line'], hue=cluster3df['late'])

### Fourth Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster4df['line'], hue=cluster4df['late'])

DBSCAN clustering algorithm was able to correctly group only late trains into the fourth cluster.

### Fifth Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster5df['line'], hue=cluster5df['late'])

### Sixth Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster6df['line'], hue=cluster6df['late'])

DBSCAN was able to correctly group late trains from the Atl. City line in this cluster.

### Seventh Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster7df['line'], hue=cluster7df['late'])

DBSCAN correctly grouped late trains again this time from the Pascack Valley and Montclair-Boonton lines.

### Eigth Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster8df['line'], hue=cluster8df['late'])

This cluster is also exclusively grouped with a true late status.

### Ninth Cluster Analysis

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(cluster9df['line'], hue=cluster9df['late'])

DBSCAN again grouped trains that were late into this cluster. In comparison to KMeans while the silhouette performance is reduced, the DBSCAN algorithm seems to be grouping trains better in regards to late status. While KMeans was grouping lines with both late and not late trains, DBSCAN seems to be grouping exclusively late or not late trains into certain clusters.

In [ ]:
# Visualizing DBSCAN clustering.
plt.figure(figsize=(11,7))
labels = dbscan_cluster.fit_predict(pca_components)
plt.scatter(PCA_components[0],PCA_components[1], c=labels); 
plt.show()

# t-SNE Dimensionality Reduction

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=90, max_iter=1000)
tsne_results = tsne.fit_transform(pca_components)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

Running and timing t-SNE. t-Distributed Stochastic Neighbor Embedding (t-SNE) is a newer method of dimensionality reduction like PCA. Unlike PCA it tries to minimize the Kullback–Leibler divergence (KL divergence) between the two dimensions which can allow for better visualization of high-dimensional data. 

### Applying KMeans to t-SNE

In [ ]:
# t-SNE reduced components stored in Pandas DataFrame.
tsne_components = pd.DataFrame(tsne_results)
# Defining range of k-values.
ks = range(1, 10)
inertias = []
for k in ks:
    # Create KMeans instance with k clusters: model.
    model = KMeans(n_clusters=k)
    # Fit model to samples.
    model.fit(tsne_components.iloc[:,:3])
    # Append the inertia to the list of inertias.
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

Let's find examine our elbow to get a feel for the best k-value before we confirm it statistically with our silhouette score. 

In [ ]:
labels = KMeans(n_clusters=2, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=3, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

Elbow method proves ineffective here as we have the highest silhouette score with a KMeans value of 3. 

In [ ]:
labels = KMeans(n_clusters=4, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=5, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=6, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=7, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=8, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=9, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
labels = KMeans(n_clusters=10, random_state=42).fit_predict(tsne_results)
print(metrics.silhouette_score(tsne_results, labels, metric='euclidean'))

In [ ]:
# Defining the k-means with the best silhouette value (3) and timing it.
kmeans_cluster = KMeans(n_clusters=3, random_state=42)

# Fit model.
%timeit kmeans_cluster.fit(tsne_results)
y_pred = kmeans_cluster.predict(tsne_results)

## KMeans Cluster Analysis with t-SNE

In [ ]:
# Attaching cluster labels to original dataframe.
df2['kmeans_tsne_cluster_label'] = kmeans_cluster.labels_

# Creating boolean arrays to pull clusters from original dataset.
cluster1 = df2['kmeans_tsne_cluster_label'] == 0
cluster2 = df2['kmeans_tsne_cluster_label'] == 1
cluster3 = df2['kmeans_tsne_cluster_label'] == 2

# Creating dataframe for each cluster found with KMeans and t-SNE.
cluster1df = df2[cluster1]
cluster2df = df2[cluster2]
cluster3df = df2[cluster3]

### First Cluster Analysis

In [ ]:
plt.figure(figsize=(17,6))
sns.countplot(cluster1df['line'], hue=cluster1df['late'])

### Second Cluster Analysis

In [ ]:
plt.figure(figsize=(17,6))
sns.countplot(cluster2df['line'], hue=cluster2df['late'])

### Third Cluster Analysis

In [ ]:
plt.figure(figsize=(17,6))
sns.countplot(cluster3df ['line'], hue=cluster3df ['late'])

In [ ]:
# Finally, let's visualize the clusters formed by KMeans and t-SNE with a simple scatterplot.
plt.figure(figsize=(10,6))
plt.scatter(tsne_components[0],tsne_components[1], c=y_pred); 
plt.show()

The clustering seen here is very simple and does not conform to the reduced dimensional shape all too much. 

## Applying DBSCAN Clustering to t-SNE

In [ ]:
dbscan_cluster = DBSCAN(eps=.1, min_samples=50)
clusters = dbscan_cluster.fit_predict(tsne_results)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(tsne_results, clusters, metric='euclidean')))

Defining the density based algorithm DBSCAN and adjusting the epsilon and minimum sample values. The silhouette score may hopefully be improved upon with different hyperparameter values as our DBSCAN clustering algorithm is performing significantly worse on t-SNE reduced features.

In [ ]:
dbscan_cluster = DBSCAN(eps=.1, min_samples=75)
clusters = dbscan_cluster.fit_predict(tsne_results)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(tsne_results, clusters, metric='euclidean')))

Increased minimum sample size made performance even worse in our new clusters. Let's try adjusting epsilon value.

In [ ]:
dbscan_cluster = DBSCAN(eps=.2, min_samples=50)
clusters = dbscan_cluster.fit_predict(tsne_results)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(tsne_results, clusters, metric='euclidean')))

Performance increased with epsilon value increase but it is still performing very poorly. Let's try increasing minimum sample value.

In [ ]:
dbscan_cluster = DBSCAN(eps=.2, min_samples=100)
clusters = dbscan_cluster.fit_predict(tsne_results)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(tsne_results, clusters, metric='euclidean')))

It seems we need to minimize our minimum sample value instead, let's try a value of 2.

In [ ]:
dbscan_cluster = DBSCAN(eps=.2, min_samples=2)
clusters = dbscan_cluster.fit_predict(tsne_results)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(tsne_results, clusters, metric='euclidean')))

Much better performance than our initial DBSCAN clusters, almost perfect clustering. Let's try a couple of other epsilon values to confirm this is the right choice.

In [ ]:
dbscan_cluster = DBSCAN(eps=.4, min_samples=2)
clusters = dbscan_cluster.fit_predict(tsne_results)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(tsne_results, clusters, metric='euclidean')))

In [ ]:
dbscan_cluster = DBSCAN(eps=.3, min_samples=2)
clusters = dbscan_cluster.fit_predict(tsne_results)
print("The silhouette score of the DBSCAN solution: {}"
      .format(metrics.silhouette_score(tsne_results, clusters, metric='euclidean')))

In conclusion, we saw the best clustering performance with DBSCAN clustering using t-SNE reduced features with tuned hyperparameter values. By tuning these we were able to transform a poor clustering algorithm into one that outperformed our inital KMeans clustering algorithm. However, the number of clusters formed is significantly higher than with KMeans.

### DBSCAN Cluster Analysis with t-SNE

In [ ]:
# Finding the unique values listed in newly created DBSCAN clusters.
np.unique(clusters)

This clustering technique would prove valuable if we could find a specific cluster to target.

In [ ]:
# Visualizing DBSCAN clustering.
plt.figure(figsize=(11,7))
labels = dbscan_cluster.fit_predict(tsne_components)
plt.scatter(tsne_components[0],tsne_components[1], c=labels); 
plt.show()

The clustering seen here conforms more to the shape of reduced dimensional t-SNE data than the KMeans clustering which conforms more to the overall structure of the data. We have defined almost 43 individual clusters with a higher silhouette score showing that DBSCAN was able to more efficiently group data than KMeans although at a higher level of complexity. Further research could include different perplexity values to attempt reducing cluster number.

# Deep Learning

In [ ]:
# Create our target variable. 
y = df['late']

# Create X values by dropping our target from DataFrame.
X = df.drop(columns=['late'])

# Use pandas get_dummies function to create dummy variables necessary for statistical interpretation (One Hot Encoding).
X = pd.get_dummies(X)

# Implementing class balancing.
sm = SMOTE(sampling_strategy='minority', random_state=42)

X, y = sm.fit_resample(X, y)

# Scaling data.
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

# Splitting data into training and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

In [ ]:
# Timing model.
start_time = time.time()
# Initiate sequential deep learning model.
model = Sequential()
# Create first dense layer, with input shape equal to DataFrame dimensions.
model.add(Dense(128, input_shape=(790,), activation="relu"))
# Creating second dense layer with sigmoid activation for classification task.
model.add(Dense(64, activation='sigmoid'))
# Creating third output layer with sigmoid activation for classification task.
model.add(Dense(1, activation="sigmoid"))
# Compiling model with adam optimizer and binary_crossentropy for clasification task.
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])
# Fitting model to training data.
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)
# Print model time.
print("--- %s seconds ---" % round(time.time() - start_time, 3))

Creating first and second dense layers with the first layer being a relu layer. The second and final output layers will be sigmoid for the classification task at hand. We will also set the optimizer to Adam an adaptive learning rate optimization algorithm. Setting verbose to 1 prints out results after each epoch to track progress.

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

The test score for our deep learning model is almost perfect and test accuracy is also showing high performance.

In [ ]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred.round())
cm = confusion_matrix(y_test, y_pred.round(), normalize=None)

print("Classification report:")
print(report)
print("Confusion matrix:")
print(cm)

Classification report shows no signs of over or underfitting and is also showing high performance for deep learning model.

# Conclusion

In conclusion a couple of supervised models stood out as top performers. This included the decision tree model with SelectKBest selected features which had .96 accuracy score and 4.3 second run time and KNN classifier using PCA feature decomposition with the best combination of run time and performance at 6 seconds and .96 accuracy score also. Decision tree classifier with PCA decomposed features performed best in terms of scoring with an accuracy score of.98 but at the cost of a longer run time of 44 seconds.

Our unsupervised learning model using DBSCAN and t-SNE reduced features was able to cluster the data very effectively with a silhouette scoring of almost perfect at .89. Cluster analysis showed finely tuned clusters numbering 43. KMeans clustering algorithms were less effective on t-SNE and PCA reduced data but by only a little using PCA. We had scoring as high as .80 on KMeans clustered data using a KMeans cluster value of 4. This shows two different effective approaches to clustering, one displaying highly selective numerous cluster numbers seen with DBSCAN and t-SNE and the other showing broad clustering with a cluster number of 4 using KMeans and PCA data.

TensorFlow deep learning models also offered comparable performance in classification report metrics to decision tree classifier with PCA but at a longer runtime of 65 seconds. Future research could reduce the deep learning model's run time by optimizing the number of epochs or adjusting batch size and minimizing the impact on scoring. 

---

# Senior Analysis: Operational Delay Prediction System

The analysis above explores delay patterns and builds initial models. Below we
apply **production-grade practices** for an operational ML system:

| Practice | Operational impact |
|---|---|
| **Pipeline** | Reproducible preprocessing for real-time scoring |
| **Stratified CV** | Ensures delay/on-time ratio is preserved in every fold |
| **Multi-metric** | Recall on delays matters — passengers need advance warning |
| **SHAP** | Operations team needs to know *which routes/times* drive delays |


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import (
    cross_validate, StratifiedKFold, train_test_split
)
from sklearn.metrics import (
    classification_report, roc_auc_score,
    ConfusionMatrixDisplay, RocCurveDisplay
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
    from portfolio_utils.data_loader import load_nj_transit
    df_s = load_nj_transit()
except Exception:
    df_s = df.copy()

# Use existing target if available, otherwise create binary delay target
if 'status' in df_s.columns:
    y_s = (df_s['status'] != 'on time').astype(int) if df_s['status'].dtype == 'object' else df_s['status']
elif 'delay_minutes' in df_s.columns:
    y_s = (df_s['delay_minutes'] > 0).astype(int)
else:
    y_s = y.copy() if 'y' in dir() else None

num_cols = df_s.select_dtypes(include=[np.number]).columns.tolist()
drop_targets = ['status', 'delay_minutes', 'stop_sequence', 'train_id']
feature_cols = [c for c in num_cols if c not in drop_targets]
X_s = df_s[feature_cols].fillna(0)

if y_s is not None and len(X_s) == len(y_s):
    print(f'Features: {X_s.shape[1]}, Samples: {X_s.shape[0]}')
    print(f'Delay rate: {y_s.mean():.1%}')
else:
    print('Using preprocessed X, y from above')
    X_s = X if 'X' in dir() else X_new
    y_s = y


In [ ]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_s, y_s, test_size=0.2, stratify=y_s, random_state=42
)

k = min(20, X_s.shape[1])
models = {
    'Decision Tree': Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=k)),
        ('clf', DecisionTreeClassifier(random_state=42)),
    ]),
    'Random Forest': Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=k)),
        ('clf', RandomForestClassifier(n_estimators=200, random_state=42)),
    ]),
    'Gradient Boosting': Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=k)),
        ('clf', GradientBoostingClassifier(random_state=42)),
    ]),
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=k)),
        ('clf', LogisticRegression(max_iter=1000, random_state=42)),
    ]),
}

scoring = {'accuracy': 'accuracy', 'precision': 'precision_weighted',
           'recall': 'recall_weighted', 'f1': 'f1_weighted', 'roc_auc': 'roc_auc'}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = {}
for name, pipe in models.items():
    cv_res = cross_validate(pipe, X_train_s, y_train_s, cv=cv,
                            scoring=scoring, n_jobs=-1)
    results[name] = {m: cv_res[f'test_{m}'].mean() for m in scoring}
    print(f'{name}: F1={results[name]["f1"]:.4f}, ROC-AUC={results[name]["roc_auc"]:.4f}')

results_df = pd.DataFrame(results).T.round(4)
results_df


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
results_df.plot.bar(ax=ax, rot=0)
ax.set_ylabel('Score')
ax.set_title('Rail Delay Model Comparison: 5-Fold Stratified CV')
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()


In [ ]:
best_name = results_df['f1'].idxmax()
best_pipe = models[best_name]
best_pipe.fit(X_train_s, y_train_s)

y_pred_s = best_pipe.predict(X_test_s)
y_proba_s = best_pipe.predict_proba(X_test_s)[:, 1]

print(f'Best model: {best_name}')
print()
print(classification_report(y_test_s, y_pred_s, target_names=['On Time', 'Delayed']))
print(f'ROC-AUC: {roc_auc_score(y_test_s, y_proba_s):.4f}')


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ConfusionMatrixDisplay.from_predictions(
    y_test_s, y_pred_s, display_labels=['On Time', 'Delayed'],
    cmap='Blues', ax=ax1
)
ax1.set_title('Confusion Matrix')
RocCurveDisplay.from_predictions(y_test_s, y_proba_s, ax=ax2, name=best_name)
ax2.plot([0, 1], [0, 1], 'k--', alpha=0.3)
ax2.set_title('ROC Curve')
plt.tight_layout()
plt.show()


## SHAP: What drives train delays?

Operations managers need to know which factors are most predictive of delays
so they can allocate resources (extra crew, schedule padding, equipment
maintenance) to the highest-impact areas.


In [ ]:
try:
    import shap
    estimator = best_pipe.named_steps['clf']
    preprocessor = Pipeline(best_pipe.steps[:-1])
    X_train_t = preprocessor.transform(X_train_s)

    if 'selector' in best_pipe.named_steps:
        mask = best_pipe.named_steps['selector'].get_support()
        feature_names = np.array(X_s.columns)[mask].tolist()
    else:
        feature_names = X_s.columns.tolist()

    sample = X_train_t[:300]
    explainer = shap.TreeExplainer(estimator, sample)
    shap_values = explainer.shap_values(sample)

    fig, ax = plt.subplots(figsize=(10, 7))
    shap.summary_plot(shap_values, sample, feature_names=feature_names,
                      max_display=15, show=False)
    plt.title('SHAP: What Drives Train Delays?')
    plt.tight_layout()
    plt.show()
except ImportError:
    print('Install shap: pip install shap')
except Exception as e:
    print(f'SHAP analysis skipped: {e}')


## Operational recommendations

**For NJ Transit Operations:**

1. **Real-time delay prediction.** Deploy the model as a microservice that scores
   upcoming departures. Push delay probability to the passenger app so riders can
   plan alternatives before arriving at the station.

2. **Resource allocation.** Use SHAP feature importance to identify which routes,
   time windows, and infrastructure segments contribute most to delays. Target
   capital investment and crew scheduling accordingly.

3. **Schedule padding.** For routes with consistently high delay probability,
   add buffer time to the published schedule. This improves on-time performance
   metrics without requiring infrastructure changes.

4. **Seasonal retraining.** Rail delays have strong seasonal patterns (weather,
   holiday travel). Retrain quarterly with recent data to maintain accuracy.

5. **Integration with clustering.** The unsupervised analysis above identifies
   route clusters with similar delay profiles. Use these clusters to design
   targeted intervention programs rather than one-size-fits-all approaches.
